Chow test for Cluster 0

In [139]:
import pandas as pd

In [140]:
cluster0 = pd.read_csv("combined_data\cluster0.csv")
cluster1 = pd.read_csv("combined_data\cluster1.csv")

In [141]:
cluster0.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time','month'],inplace=True)
cluster1.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time','month'],inplace=True)

cluster0.dropna(inplace=True)
cluster1.dropna(inplace=True)

The models in each of the models (pooled, 1, 2) must have normally distributed error with mean 0, as well as independent and identically distributed errors, to satisfy the Gauss-Markov assumptions.


= constant mean and variance without serial correlation in the residuals. 

In [142]:
cluster0['Date'] = pd.to_datetime(cluster0['Date'])
X = cluster0.drop(columns =['Price',"Date"] )
y = cluster0['Price']

In [143]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
residuals = results.resid



# tests for heteroskedasticity in the full-sample residuals
print('F-statistic for the Breusch-Pagan Test the OLS model: ' + 
      str(het_breuschpagan(residuals, X)[2]))
print('p-value for the Breusch-Pagan F-Test the OLS model: ' + 
      str(het_breuschpagan(residuals, X)[3]))


F-statistic for the Breusch-Pagan Test the OLS model: 5.386733972075286
p-value for the Breusch-Pagan F-Test the OLS model: 7.798736684213723e-08


les résidus ont de  l'hétéroskédaticité -> utiliser HAC

In [144]:
from scipy.stats import normaltest

# performs the normality test on the residuals from the non-differenced OLS model
print(normaltest(residuals))

NormaltestResult(statistic=np.float64(0.851623701784599), pvalue=np.float64(0.6532392369385829))


et suit une loi normale

In [157]:

# Split the data
breakpoint = pd.Timestamp('2011-12-01')
df_pre = pd.DataFrame(cluster0[cluster0['Date'] < breakpoint])
df_post = pd.DataFrame(cluster0[cluster0['Date'] >= breakpoint])

In [146]:
import statsmodels.api as sm


X_pre = df_pre.drop(columns=['Date','Price'])
X_pre = sm.add_constant(X_pre)  # Using index as a regressor for simplicity
Y_pre = df_pre['Price']
model_pre = sm.OLS(Y_pre, X_pre).fit()

X_post = df_post.drop(columns=['Date','Price'])
X_post = sm.add_constant(X_post)  # Reset index for post model
Y_post = df_post['Price']
model_post = sm.OLS(Y_post, X_post).fit()

#gérer l''hétéroscédasticité 
T_pre = len(Y_pre)
T_post = len(Y_post)
maxlags_pre = int(4 * (T_pre / 100) ** (2 / 9))
maxlags_post = int(4 * (T_post / 100) ** (2 / 9))

robust_model_pre = model_pre.get_robustcov_results(cov_type='HAC', maxlags=maxlags_pre)
robust_model_post = model_post.get_robustcov_results(cov_type='HAC', maxlags=maxlags_post)


In [147]:
def ChowTest(model_pre, model_post, df_pre, df_post, total_df):
    RSS0 = sum((total_df['Price'] - total_df['Price'].mean())**2)
    RSS1 = sum((df_pre['Price'] - model_pre.predict(X_pre))**2)
    RSS2 = sum((df_post['Price'] - model_post.predict(X_post))**2)
    n = len(total_df)
    k = model_pre.params.size  # Number of parameters estimated, typically includes the constant
    F = ((RSS0 - RSS1 - RSS2) / k) / ((RSS1 + RSS2) / (n - 2 * k))
    return F

# Perform Chow Test
F_statistic= ChowTest(robust_model_pre,robust_model_post,df_pre,df_post,cluster0)
print(f"Chow Test F-statistic: {F_statistic}")


Chow Test F-statistic: 26.304277824289944


In [148]:
from scipy.stats import f

n = len(cluster0)  # total number of observations
k =  X_pre.shape[1] + 1  # number of parameters estimated in one model

# Degrees of freedom
dfn = k  # number of parameters estimated
dfd = n - 2 * k  # total observations minus twice the number of estimated parameters

# Calculating the p-value from the F-distribution
p_value = 1 - f.cdf(F_statistic, dfn, dfd)
print(f"Chow Test p-value: {p_value}")


Chow Test p-value: 1.1102230246251565e-16


Oui il y a un breakthrough en dec 2011

REFAIRE POUR CLUSTER 1

In [149]:
cluster1['Date'] = pd.to_datetime(cluster1['Date'])
X = cluster1.drop(columns =['Price',"Date"] )
y = cluster1['Price']

In [150]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
residuals = results.resid

# tests for heteroskedasticity in the full-sample residuals
print('F-statistic for the Breusch-Pagan Test the OLS model: ' + 
      str(het_breuschpagan(residuals, X)[2]))
print('p-value for the Breusch-Pagan F-Test the OLS model: ' + 
      str(het_breuschpagan(residuals, X)[3]))


F-statistic for the Breusch-Pagan Test the OLS model: 5.093690010082673
p-value for the Breusch-Pagan F-Test the OLS model: 2.3175660951247766e-07


les résidus ont de  l'hétéroskédaticité -> utiliser HAC

In [151]:
from scipy.stats import normaltest

# performs the normality test on the residuals from the non-differenced OLS model
print(normaltest(residuals))

NormaltestResult(statistic=np.float64(1.4136116201345006), pvalue=np.float64(0.4932171131653922))


et suit une loi normale

In [152]:

# Split the data
breakpoint = pd.Timestamp('2011-12-01')
df_pre = pd.DataFrame(cluster1[cluster1['Date'] < breakpoint])
df_post = pd.DataFrame(cluster1[cluster1['Date'] >= breakpoint])

In [153]:
import statsmodels.api as sm


X_pre = df_pre.drop(columns=['Date','Price'])
X_pre = sm.add_constant(X_pre)  # Using index as a regressor for simplicity
Y_pre = df_pre['Price']
model_pre = sm.OLS(Y_pre, X_pre).fit()

X_post = df_post.drop(columns=['Date','Price'])
X_post = sm.add_constant(X_post)  # Reset index for post model
Y_post = df_post['Price']
model_post = sm.OLS(Y_post, X_post).fit()

#gérer l''hétéroscédasticité 
T_pre = len(Y_pre)
T_post = len(Y_post)
maxlags_pre = int(4 * (T_pre / 100) ** (2 / 9))
maxlags_post = int(4 * (T_post / 100) ** (2 / 9))

robust_model_pre = model_pre.get_robustcov_results(cov_type='HAC', maxlags=maxlags_pre)
robust_model_post = model_post.get_robustcov_results(cov_type='HAC', maxlags=maxlags_post)


In [154]:
F_statistic = ChowTest(model_pre,model_post,df_pre,df_post,cluster1)

In [155]:
from scipy.stats import f

n = len(cluster1)  # total number of observations
k =  X_pre.shape[1] + 1  # number of parameters estimated in one model

# Degrees of freedom
dfn = k  # number of parameters estimated
dfd = n - 2 * k  # total observations minus twice the number of estimated parameters

# Calculating the p-value from the F-distribution
p_value = 1 - f.cdf(F_statistic, dfn, dfd)
print(f"Chow Test p-value: {p_value}")


Chow Test p-value: 1.1102230246251565e-16
